In [1]:
import pandas as pd
import yfinance as yf
from yahoo_fin import stock_info
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [5]:
def get_percentage(input, min, max):
    return round((input - min) / (max - min) * 100)

def calculate_buysell_score(current_div_yield, div_min, div_max):
    buy_score = get_percentage(current_div_yield, div_min, div_max)
    sell_score = 100 - buy_score
    return buy_score, sell_score

In [28]:
class UsaStockDividendAnalyzer:
    def __init__(self, ticker, period):
        self.ticker = ticker
        self.period = period

    def fit(self):
        ticker = self.ticker
        period = self.period

        print(f'ticker = {ticker}')
        stock_basic_info = yf.Ticker(ticker).info
        end_date = datetime.now().strftime("%Y-%m-%d")
        #print("start date: " , end_date)
        start_date = (datetime.now() - relativedelta(years=period)).strftime("%Y-%m-%d")
        #print("end date: ", start_date)

        dividends = stock_info.get_dividends(ticker, start_date=start_date, end_date=end_date)

        stock_data = yf.download(ticker, start_date, end_date)

        df_close_price = stock_data.loc[dividends.index]['Close']

        dividends['close'] = df_close_price

        dividends = dividends.drop('ticker', axis=1)

        div_freq = round(len(dividends.index)/period)

        dividends['div yield'] = round(dividends['dividend']*div_freq / dividends['close'] * 100, 2)

        div_min = min(dividends['div yield'])
        div_max = max(dividends['div yield'])

        buy_price = dividends['dividend'][-1] * div_freq * 100 / div_min
        sell_price = dividends['dividend'][-1] * div_freq * 100 / div_max

        current_datetime = datetime.strptime(end_date, "%Y-%m-%d")
        current_price = stock_data.iloc[-1]['Close']

        last_dividend = dividends.iloc[-1]['dividend']
        self.current_dividend = last_dividend * div_freq
        self.current_div_yield = round(self.current_dividend/current_price*100, 2)

        self.buy_score, self.sell_score = calculate_buysell_score(self.current_div_yield, div_min, div_max)
        #print(f'buy score = {buy_score}\nsell score = {sell_score}')
        
        
    def getBuyScore(self):
        return self.buy_score

    def getSellScore(self):
        return self.sell_score
    
    def getCurrentDividend(self):
        return self.current_dividend
    
    def getCurrentDividendYield(self):
        return self.current_div_yield

In [29]:
DIV_PERIODS = 7

In [64]:
analyzer = UsaStockDividendAnalyzer('PFE', DIV_PERIODS)

In [65]:
analyzer.fit()

ticker = PFE
[*********************100%***********************]  1 of 1 completed


In [66]:
analyzer.getBuyScore()

-4

In [67]:
analyzer.getSellScore()

104

In [68]:
analyzer.getCurrentDividend()

1.6

In [69]:
analyzer.getCurrentDividendYield()

2.94

In [37]:
stock_df = pd.read_csv('usa-dividend-stock-list.csv')
stock_df.head(3)

,ticker
0,AMT
1,DLR
2,KMB


In [48]:
dividends = []
dividend_yields = []
buy_scores = []
sell_scores= []
for ticker in stock_df['ticker']:
    analyzer = UsaStockDividendAnalyzer(ticker, DIV_PERIODS)
    analyzer.fit()
    dividends.append(analyzer.getCurrentDividend())
    dividend_yields.append(analyzer.getCurrentDividendYield())
    buy_scores.append(analyzer.getBuyScore())



ticker = AMT
[*********************100%***********************]  1 of 1 completed
ticker = DLR
[*********************100%***********************]  1 of 1 completed
ticker = KMB
[*********************100%***********************]  1 of 1 completed
ticker = MCD
[*********************100%***********************]  1 of 1 completed
ticker = MRK
[*********************100%***********************]  1 of 1 completed
ticker = NEE
[*********************100%***********************]  1 of 1 completed
ticker = O
[*********************100%***********************]  1 of 1 completed
ticker = OHI
[*********************100%***********************]  1 of 1 completed
ticker = VZ
[*********************100%***********************]  1 of 1 completed
ticker = INTC
[*********************100%***********************]  1 of 1 completed
ticker = WPC
[*********************100%***********************]  1 of 1 completed
ticker = LMT
[*********************100%***********************]  1 of 1 completed
ticker = MSFT
[***

KeyError: "['ticker'] not found in axis"

In [49]:
dividends

[5.88,
 4.88,
 4.64,
 6.08,
 2.92,
 1.7,
 2.976,
 2.68,
 2.612,
 1.46,
 4.244,
 12.0,
 2.72,
 0.92,
 4.96,
 0.28,
 4.52,
 1.6,
 2.48,
 1.52,
 3.4,
 6.6,
 1.112]

In [50]:
dividend_yields

[2.67,
 4.66,
 3.34,
 2.21,
 2.62,
 1.95,
 4.6,
 9.1,
 6.97,
 5.17,
 5.28,
 2.47,
 1.06,
 0.64,
 2.82,
 0.98,
 2.51,
 2.27,
 2.56,
 3.08,
 0.96,
 4.4,
 5.88]

In [51]:
buy_scores

[73,
 100,
 48,
 8,
 0,
 14,
 56,
 68,
 102,
 100,
 28,
 18,
 16,
 7,
 57,
 61,
 21,
 20,
 92,
 50,
 30,
 50,
 10]

In [52]:
result_df = pd.DataFrame({'annual dividend':dividends, 
                          'dividend yield': dividend_yields,
                          'buy score': buy_scores})

In [56]:
final_df = pd.concat([stock_df, result_df], axis=1)

In [58]:
final_df.head(3)

,ticker,annual dividend,dividend yield,buy score
0,AMT,5.88,2.67,73.0
1,DLR,4.88,4.66,100.0
2,KMB,4.64,3.34,48.0


In [63]:
final_df.to_excel('DIVIDEND-TABLE.xlsx', index=False)
final_df.to_csv('DIVIDEND-TABLE.csv', index=False)